In [1]:
import sys
import math
import csv
import random
import json 
import time

try:
    jparams = json.load(open('params.json'))
except:
    print("ERROR: something is wrong with the params.json file.")
    sys.exit()
    
#-- store the input 3D points in list
list_pts_3d = []
with open(jparams['input-file']) as csvfile:
    r = csv.reader(csvfile, delimiter=' ')
    header = next(r)
    for line in r:
        p = list(map(float, line)) #-- convert each str to a float
        assert(len(p) == 3)
        list_pts_3d.append(p)

In [2]:
import math
import numpy
import scipy.spatial

In [3]:
class BoundingBox:
    """A 2D bounding box"""
    
    def __init__(self, points):
        if len(points) == 0:
            raise ValueError("Can't compute bounding box of empty list")
        
        self.minx, self.miny = float("inf"), float("inf")
        self.maxx, self.maxy = float("-inf"), float("-inf")
        for x, y, *_ in points:
            # Set min coords
            if x < self.minx:
                self.minx = x
            if y < self.miny:
                self.miny = y
            # Set max coords
            if x > self.maxx:
                self.maxx = x
            elif y > self.maxy:
                self.maxy = y
                
    @property
    def width(self):
        return self.maxx - self.minx
    @property
    def height(self):
        return self.maxy - self.miny
    def __repr__(self):
        return "BoundingBox(X: {} - {}, Y: {} - {})".format(
            self.minx, self.maxx, self.miny, self.maxy)
      
bbox = BoundingBox(list_pts_3d)
bbox

BoundingBox(X: 2.0 - 252.0, Y: 3.0 - 253.0)

In [42]:
class Raster:
  """Simple raster based on ESRI ASCII schema"""
  
  def __init__(self, bbox, cell_size, no_data=-9999):
    self.ncols = int(bbox.width // cell_size + 1)
    self.nrows = int(bbox.height // cell_size + 1)
    self.xllcenter = bbox.minx
    self.yllcenter = bbox.miny
    self.cell_size = cell_size
    self.no_data = no_data
    
    # initialize list of values with no_data
    self.values = [self.no_data] * self.ncols * self.nrows
  
  @property
  def coords(self):
    """Cell center coordinates"""
    for i in range(self.ncols):
      for j in range(self.nrows):
        x = self.xllcenter + i * self.cell_size
        y = self.yllcenter + j * self.cell_size
        yield (x,y)
    
  def to_ascii(self):
    rows = [
      "NCOLS %d" % self.ncols,
      "NROWS %d" % self.nrows,
      "XLLCENTER %s" % self.xllcenter,
      "YLLCENTER %s" % self.yllcenter,
      "CELLSIZE %s" % self.cell_size,
      "NODATA_VALUE %s" % self.no_data,
      ' '.join( (str(x) for x in self.values) )
    ]
    return '\n'.join(rows)

In [17]:
bbox = BoundingBox(list_pts_3d)
raster = Raster(bbox, 30)
type(raster.to_ascii())

str

In [29]:
list_points_2d = [(x,y) for x,y,z in list_pts_3d]
list_points_z = [(z) for x,y,z in list_pts_3d]

kdtree = scipy.spatial.KDTree(list_points_2d)

for i, coord in enumerate(raster.coords):
    dist, index = kdtree.query(coord)
    raster.values[i] = list_points_z[index]


[70.0, 70.305275, 84.47369, 101.77487, 111.0, 110.0, 101.35417, 86.04014, 72.97872, 70.0, 79.0, 90.38461, 108.81818, 116.50589, 116.0, 101.11111, 79.642395, 62.290043, 70.25, 92.71739, 107.87879, 121.3, 132.78226, 128.60825, 114.67391, 82.75862, 60.0, 84.324326, 107.88889, 126.63462, 136.07954, 145.9806, 138.61479, 123.02326, 105.2381, 76.92308, 87.65306, 109.54545, 131.66667, 148.43137, 147.07472, 115.92593, 113.05556, 105.56053, 91.8421, 97.3935, 116.33333, 139.7999, 136.72414, 123.5, 101.85185, 90.55556, 89.09091, 75.333336, 115.2881, 129.19017, 130.0, 112.14286, 103.75, 89.72222, 65.005684, 50.454544, 48.214287, 120.86963, 127.64343, 121.70886, 96.07143, 65.125, 65.90909, 62.350628, 45.87662, 37.227367, 110.75431, 107.25723, 107.16216, 78.888885, 52.820515, 44.14634, 43.82353, 41.69956, 33.60824]
